# Notebook 3: Funções de Transformação e Limpeza de Dados

## Introdução
Neste notebook, iremos implementar as transformações e limpar os dados da tabela 'Reaction' do conjunto de dados FAERS, com base na análise exploratória realizada no notebook anterior (Exploratory_Data_Analysis_Reaction). 

## Estrutura do notebook
- Importação de Bibliotecas
- Carregamento dos Dados (Parquet Raw) usando o Schema definido 
- Aplicar apenas as Transformações necessárias
- Salvar **Parquet Final** com os Dados Limpos


In [0]:
# Importar bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, LongType

In [0]:
%run "../Funcoes_auxiliares" 

In [0]:
# Carregar dados do Parquet raw
schema_reac = StructType([
    StructField("primaryid", LongType(), False,{'description': "Unique number for identifying a FAERS report."}),
    StructField("caseid", IntegerType(), False,{'description': "Number for identifying a FAERS case."}),
    StructField("pt", StringType(), False,{'description': "'Preferred Term'-level medical terminology describing the event, using the Medical Dictionary for Regulatory Activities (MedDRA)."}),
    StructField("drug_rec_act", StringType(), True, {'description': "Drug Recur Action data - populated with reaction/event the event reappears upon readministration of the drug."})
])
df_reac = spark.read.format('parquet').schema(schema_reac).load('dbfs:/FileStore/FAERS-grupo-4/reac_raw',mode="PERMISSIVE")

In [0]:
#Transformar a coluna 'drug_rec_act' em Boleano
df_reac = df_reac.withColumn('drug_rec_act_bool', df_reac['drug_rec_act'].isNotNull())
#Eliminar a coluna original
df_reac=df_reac.drop('drug_rec_act')
#Mudar nome das colunas
df_reac=df_reac.withColumnsRenamed({'pt':'Event','drug_rec_act_bool':'Recur_Event'})

In [0]:
#Restaurar metadados
df_reac = restore_metadata(schema_reac,df_reac)

In [0]:
# salvar parquet final
df_reac.write.mode('overwrite').parquet("dbfs:/FileStore/FAERS-grupo-4/reac_final")